In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:
pl.__version__

In [ ]:
size = 10_000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)
df.head()

In [ ]:
%%timeit
df.filter(
    pds.query_knn_filter("var1", "var2", "var3", pt = [0.5, 0.5, 0.5], k = 3)
)